In [26]:
import torch
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler

import torch.nn as nn
import matplotlib.pyplot as plt

In [27]:
# Step 1: Data Preparation
# Download sample stock data (using Apple stock as an example)
def get_stock_data(symbol='AAPL', start='2020-01-01', end='2023-12-31'):
    """Download stock data using yfinance"""
    stock = yf.download(symbol, start=start, end=end)
    return stock['Close'].values.reshape(-1, 1)

In [28]:
# Step 2: Create sequences for time series prediction
def create_sequences(data, seq_length):
    """
    Convert time series data into sequences
    Input: [1, 2, 3, 4, 5] and seq_length=3
    Output: X=[[1,2,3], [2,3,4]], y=[[4], [5]]
    """
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:(i + seq_length)])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

In [29]:
# Step 3: Define the GRU Model
class StockPredictor(nn.Module):
    def __init__(self, input_size=1, hidden_size=32, num_layers=2):
        """
        Initialize the GRU model
        input_size: number of features (1 for single stock price)
        hidden_size: number of GRU units
        num_layers: number of GRU layers
        """
        super(StockPredictor, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # GRU layer
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        # Fully connected layer for output
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        """Forward pass through the network"""
        # Initialize hidden state
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)

        # Forward propagate GRU
        out, _ = self.gru(x, h0)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

In [30]:
# Step 4: Training function
def train_model(model, X_train, y_train, epochs=100, lr=0.01):
    """Train the GRU model"""
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        # Forward pass
        outputs = model(X_train)
        loss = criterion(outputs, y_train)

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

In [31]:
def main():
    # Get stock data
    data = get_stock_data()

    # Normalize data
    scaler = MinMaxScaler()
    data_normalized = scaler.fit_transform(data)

    # Create sequences
    seq_length = 10
    X, y = create_sequences(data_normalized, seq_length)

    # Convert to PyTorch tensors
    X_train = torch.FloatTensor(X)
    y_train = torch.FloatTensor(y)

    # Initialize and train model
    model = StockPredictor()
    train_model(model, X_train, y_train)

    # Make predictions
    model.eval()
    with torch.no_grad():
        test_sequence = X_train[-1].unsqueeze(0)
        prediction = model(test_sequence)
        prediction = scaler.inverse_transform(prediction.numpy())
        print(f"Next day's predicted price: ${prediction[0][0]:.2f}")

if __name__ == "__main__":
    main()

[*********************100%***********************]  1 of 1 completed


Epoch [10/100], Loss: 0.0439
Epoch [20/100], Loss: 0.0127
Epoch [30/100], Loss: 0.0036
Epoch [40/100], Loss: 0.0021
Epoch [50/100], Loss: 0.0011
Epoch [60/100], Loss: 0.0009
Epoch [70/100], Loss: 0.0007
Epoch [80/100], Loss: 0.0006
Epoch [90/100], Loss: 0.0006
Epoch [100/100], Loss: 0.0006
Next day's predicted price: $190.68
